# attend-dash
 Currently this notebook is divided into parts. Run only the parts you need. Run the first part the first time you open this notebook.

## 1. Scraping

In [16]:
import requests

In [17]:
url = "http://apf.ststephens.edu/attendance_stku2018"

payload = {
'programme':'33',
'semester':'10',
'month':'07'
}

In [18]:
r = requests.post(url, data=payload)


In [20]:
f = open('response.html', 'w')
f.write(r.text)

631688

## 2. Cleaning

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [195]:
pd_table = pd.read_html('response.html',header=1)[0]
pd_table.drop(columns=pd_table.columns.values.tolist()[-11:], inplace=True)
pd_table.head()

,LD,LA,TH,TA,PH,PA,LD.1,LA.1,TH.1,TA.1,...,TA.10,PH.10,PA.10,LD.11,LA.11,TH.11,TA.11,PH.11,PA.11,LD.12
0,Adya Singh,5.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,3.0,0.0,0.0,5.0,5.0
1,Mansi Puggal,5.0,4.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,3.0,0.0,0.0,5.0,5.0
2,K Chungneithang Kom,NaN,NaN,NaN,NaN,NaN,NaN,7.0,6.0,0.0,...,NaN,NaN,NaN,NaN,3.0,3.0,0.0,0.0,5.0,5.0
3,Niranjan Palaparthi,NaN,NaN,NaN,NaN,NaN,NaN,7.0,7.0,0.0,...,NaN,NaN,NaN,NaN,3.0,3.0,0.0,0.0,5.0,5.0
4,Divyapreet Kaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,3.0,0.0,0.0,5.0,5.0


In [197]:
f = open('response.html', 'r') 

soup = BeautifulSoup(f.read(),'html.parser')
def is_paper_name(colspan):
    ''' Paper names have headers which are 6 columns wide '''
    return colspan and colspan == '6'

headers = [paper_name.text.strip() for paper_name  in soup.find_all(colspan = is_paper_name)]
headers

['Media and  Communications GE II',
 'Making of Contemporary India',
 'Indian Economy - I',
 'Money and Banking',
 'Computer Networks and Internet Technologies',
 'Theory of Functions',
 'Group Theory - I',
 'Multivariate Calculus',
 'Multivariate Calculus Group I',
 'Multivariate Calculus Group II',
 'Latex and HTML SEC Group I',
 'Latex and HTML SEC Group II']

In [198]:
paper_cols = headers
types = ['Lectures_Held', 'Lectures_Attended',
         'Tutorials_Held','Tutorials_Attended',
         'Practicals_Held', 'Practicals_Attended',]
iterables = [paper_cols, types]
full_index = pd.MultiIndex.from_product(iterables)
full_index = full_index.values.tolist()

In [200]:
full_index[0]

('Media and  Communications GE II', 'Lectures_Held')

In [201]:
def join_attendance(heading_tuple):
    return '_'.join(heading_tuple)

In [202]:
modified_col_names = list(map(join_attendance, full_index))
modified_col_names.insert(0,'Name')
col_names = pd_table.columns.values.tolist()
col_rename = dict(zip(col_names,modified_col_names))
pd_table.rename(mapper=col_rename, axis=1,inplace=True)
pd_table.rename_axis('Name',axis=0, inplace=True)
pd_table.head()

,Name,Media and Communications GE II_Lectures_Held,Media and Communications GE II_Lectures_Attended,Media and Communications GE II_Tutorials_Held,Media and Communications GE II_Tutorials_Attended,Media and Communications GE II_Practicals_Held,Media and Communications GE II_Practicals_Attended,Making of Contemporary India_Lectures_Held,Making of Contemporary India_Lectures_Attended,Making of Contemporary India_Tutorials_Held,...,Latex and HTML SEC Group I_Tutorials_Held,Latex and HTML SEC Group I_Tutorials_Attended,Latex and HTML SEC Group I_Practicals_Held,Latex and HTML SEC Group I_Practicals_Attended,Latex and HTML SEC Group II_Lectures_Held,Latex and HTML SEC Group II_Lectures_Attended,Latex and HTML SEC Group II_Tutorials_Held,Latex and HTML SEC Group II_Tutorials_Attended,Latex and HTML SEC Group II_Practicals_Held,Latex and HTML SEC Group II_Practicals_Attended
Name,,,,,,,,,,,,,,,,,,,,,
0,Adya Singh,5.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,3.0,0.0,0.0,5.0,5.0
1,Mansi Puggal,5.0,4.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,3.0,0.0,0.0,5.0,5.0
2,K Chungneithang Kom,NaN,NaN,NaN,NaN,NaN,NaN,7.0,6.0,0.0,...,NaN,NaN,NaN,NaN,3.0,3.0,0.0,0.0,5.0,5.0
3,Niranjan Palaparthi,NaN,NaN,NaN,NaN,NaN,NaN,7.0,7.0,0.0,...,NaN,NaN,NaN,NaN,3.0,3.0,0.0,0.0,5.0,5.0
4,Divyapreet Kaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,3.0,0.0,0.0,5.0,5.0


In [218]:
pd_melted = pd.melt(pd_table,id_vars=['Name'], var_name='Paper_Attendance')
paper_types = pd_melted['Paper_Attendance'].str.split('_')

In [219]:
paper_names = paper_types.str.get(0)
class_type = paper_types.str.get(1)
attendance_type = paper_types.str.get(2)


In [221]:
pd_melted['Paper Name'] = paper_names
pd_melted['Class Type'] = class_type
pd_melted['Attendance Type'] = attendance_type
pd_melted.drop('Paper_Attendance', axis =1, inplace=True)

In [223]:
pd_melted.head()

,Name,value,Paper Name,Class Type,Attendance Type
0,Adya Singh,5.0,Media and Communications GE II,Lectures,Held
1,Mansi Puggal,5.0,Media and Communications GE II,Lectures,Held
2,K Chungneithang Kom,NaN,Media and Communications GE II,Lectures,Held
3,Niranjan Palaparthi,NaN,Media and Communications GE II,Lectures,Held
4,Divyapreet Kaur,NaN,Media and Communications GE II,Lectures,Held


In [228]:
attend_tidy = pd_melted.pivot_table(
    index=['Name', 'Paper Name', 'Class Type'],
    columns='Attendance Type',
    values='value'
)
attend_tidy

Attendance Type                                                      Attended  \
Name         Paper Name                                  Class Type             
Adya Singh   Computer Networks and Internet Technologies Lectures         NaN   
                                                         Practicals       NaN   
                                                         Tutorials        NaN   
             Group Theory - I                            Lectures         8.0   
                                                         Practicals       0.0   
                                                         Tutorials        0.0   
             Indian Economy - I                          Lectures         NaN   
                                                         Practicals       NaN   
                                                         Tutorials        NaN   
             Latex and HTML SEC Group I                  Lectures         NaN   
                                                         Practicals       NaN   
                                                         Tutorials        NaN   
             Latex and HTML SEC Group II                 Lectures         3.0   
                                                         Practicals       5.0   
                                                         Tutorials        0.0   
             Making of Contemporary India                Lectures         NaN   
                                                         Practicals       NaN   
                                                         Tutorials        NaN   
             Media and  Communications GE II             Lectures         0.0   
                                                         Practicals       0.0   
                                                         Tutorials        0.0   
             Money and Banking                           Lectures         NaN   
                                                         Practicals       NaN   
                                                         Tutorials        NaN   
             Multivariate Calculus                       Lectures         5.0   
                                                         Practicals       0.0   
                                                         Tutorials        0.0   
             Multivariate Calculus Group I               Lectures         NaN   
                                                         Practicals       NaN   
                                                         Tutorials        NaN   
...                                                                       ...   
Yash Lazarus Indian Economy - I                          Lectures         NaN   
                                                         Practicals       NaN   
                                                         Tutorials        NaN   
             Latex and HTML SEC Group I                  Lectures         3.0   
                                                         Practicals       7.0   
                                                         Tutorials        0.0   
             Latex and HTML SEC Group II                 Lectures         NaN   
                                                         Practicals       NaN   
                                                         Tutorials        NaN   
             Making of Contemporary India                Lectures         NaN   
                                                         Practicals       NaN   
                                                         Tutorials        NaN   
             Media and  Communications GE II             Lectures         NaN   
                                                         Practicals       NaN   
                                                         Tutorials        NaN   
             Money and Banking                           Lectures         5.0   
                               

In [239]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\abhis\.virtualenvs\attend-dash-PlWwuj0B\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


And thus we have computer the total attendance across groups. The attendance is in the folowing format:

| Lectured Delivered | Lectures Attended | Tutorials Held | Tutorials Attended | Practicals Held | Practicals Attended |
| --- | --- | --- | --- | --- | --- |



### TODO
1. Scrape individual attendance data per student 
2. Calculate necessary values for aggregate calculations and store them in dataframe or np.array
3. Prepare data to display for individual summary and aggregate summary

### Potential individual summary format
- name
- number of papers
- for each paper
    - paper name
    - paper attendance
- total attendance